<a href="https://colab.research.google.com/github/zhr-frj/daftarchePhd/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Nov 12 09:48:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# --- سلول ۱: نصب استاندارد و پایدار ---
print("--- در حال نصب کتابخانه‌های استاندارد ---")
!pip install langchain langchain-community transformers bitsandbytes accelerate faiss-cpu pypdf sentence-transformers langchain_huggingface docarray -q
print("--- نصب کامل شد ---")

--- در حال نصب کتابخانه‌های استاندارد ---
--- نصب کامل شد ---


In [16]:
# --- وارد کردن ابزارها و تنظیمات فشرده‌سازی ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

print("--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---")
# تنظیمات فشرده‌سازی 4-bit برای کاهش مصرف حافظه
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)
print("--- ابزارها و تنظیمات آماده شدند ---")

--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---
--- ابزارها و تنظیمات آماده شدند ---


In [17]:
# # --- بارگذاری «مترجم» (Tokenizer) ---
# print("--- در حال بارگذاری «مترجم» (Tokenizer) ---")
# tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-9b-it", use_fast=True)
# print("--- «مترجم» (Tokenizer) بارگذاری شد ---")


# --- سلول ۲.۲: بارگذاری «مترجم» (Tokenizer) - نسخه سبک Gemma 2B ---
from transformers import AutoTokenizer

print("--- در حال بارگذاری «مترجم» (Gemma 2B) ---")
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/gemma-2-2b-it",
    use_fast=True
    )
print("--- «مترجم» (Tokenizer) بارگذاری شد ---")

--- در حال بارگذاری «مترجم» (Gemma 2B) ---
--- «مترجم» (Tokenizer) بارگذاری شد ---


In [18]:
# # --- بارگذاری «مغز» (LLM) ---
# print("--- در حال بارگذاری «مغز» (LLM) --- (این مرحله بسیار سنگین و زمان‌بر است)")
# model = AutoModelForCausalLM.from_pretrained(
#     "unsloth/gemma-2-9b-it",
#     device_map="auto",
#     quantization_config=bnb_config,
#     torch_dtype=torch.float16
# )
# print("--- «مغز» (LLM) با موفقیت بارگذاری شد ---")



# --- سلول ۲.۳: بارگذاری «مغز» (LLM) - نسخه سبک Gemma 2B ---
from transformers import AutoModelForCausalLM

print("--- در حال بارگذاری «مغز» (LLM) --- (نسخه سبک Gemma 2B)")
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2-2b-it",
    device_map="auto",
    quantization_config=bnb_config, # از bnb_config استفاده می‌کند
    torch_dtype=torch.float16
)
print("--- «مغز» (LLM) با موفقیت بارگذاری شد ---")

--- در حال بارگذاری «مغز» (LLM) --- (نسخه سبک Gemma 2B)
--- «مغز» (LLM) با موفقیت بارگذاری شد ---


In [19]:
# --- ساخت «خط لوله» و ابزارهای LangChain ---
print("--- در حال ساخت «خط لوله» (Pipeline) ---")
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512  # افزایش طول پاسخ برای جواب‌های کامل‌تر
)

# ساخت «غلاف» LangChain برای LLM
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# ساخت «تمیزکننده خروجی»
parser = StrOutputParser()

print("--- «خط لوله»، «غلاف» و «تمیزکننده» آماده شدند ---")

Device set to use cuda:0


--- در حال ساخت «خط لوله» (Pipeline) ---
--- «خط لوله»، «غلاف» و «تمیزکننده» آماده شدند ---


In [20]:
# --- تابع کمکی برای اعمال «قالب چت» ---

# این تابع، پرامپت نهایی ما را در قالب چت رسمی مدل Gemma قرار می‌دهد
def apply_chat_template_and_response(prompt):
    messages = [
    {'role': 'user', 'content': prompt}
    ]

    # اعمال قالب چت
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    # فراخوانی مدل و تمیزکاری اولیه
    response = llm.invoke(text)
    # response را تمیز می‌کنیم تا فقط جواب نهایی باقی بماند
    return response.replace(text, '').strip()

print("--- تابع کمکی «قالب چت» آماده شد ---")

--- تابع کمکی «قالب چت» آماده شد ---


In [21]:
# --- بارگذاری «بینی معنی‌فهم» (Embedding Model) ---

from langchain_huggingface import HuggingFaceEmbeddings

print("--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) ---")
# ما از یک مدل امبدینگ فارسی قوی استفاده می‌کنیم
embedding_name = 'sbunlp/fabert'
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name,
    model_kwargs={'device': 'cpu'} # امبدینگ را روی CPU اجرا می‌کنیم تا حافظه GPU برای LLM آزاد بماند
)
print("--- «بینی معنی‌فهم» (Embeddings) آماده شد ---")

--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) ---


Some weights of BertModel were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- «بینی معنی‌فهم» (Embeddings) آماده شد ---


In [22]:
# # --- تعریف «برگه دستورالعمل» (Prompt Template) ---

# from langchain.prompts import PromptTemplate

# # این الگو به ربات می‌گوید که «خارج از PDF» جواب ندهد
# template = """
# شما یک دستیار هوشمند و بسیار دقیق هستید. وظیفه شما پاسخ به سوال کاربر فقط و فقط بر اساس متنی است که در بخش «زمینه» به شما داده می‌شود.
# به هیچ وجه از دانش قبلی خود استفاده نکنید.
# پاسخ را فقط به زبان فارسی ارائه دهید.
# اگر پاسخ سوال در «زمینه» وجود نداشت، به سادگی بگویید: "متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم."

# زمینه (متن‌های یافت‌شده از PDF):
# {context}

# سوال کاربر:
# {question}

# پاسخ دقیق شما:
# """

# prompt = PromptTemplate.from_template(template)
# print("--- «برگه دستورالعمل» (Prompt) آماده شد ---")





# --- سلول ۵: تعریف «برگه دستورالعمل» (Prompt Template) - نسخه Gemma ---
from langchain.prompts import PromptTemplate

# قالب پرامپت استاندارد (بدون تگ‌های <|user|>)
template = """
شما یک دستیار هوشمند و بسیار دقیق هستید. وظیفه شما پاسخ به سوال کاربر فقط و فقط بر اساس متنی است که در بخش «زمینه» به شما داده می‌شود.
به هیچ وجه از دانش قبلی خود استفاده نکنید.
پاسخ را فقط به زبان فارسی ارائه دهید.
اگر پاسخ سوال در «زمینه» وجود نداشت، به سادگی بگویید: "متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم."

زمینه (متن‌های یافت‌شده از PDF):
{context}

سوال کاربر:
{question}

پاسخ دقیق شما:
"""

prompt = PromptTemplate.from_template(template)
print("--- «برگه دستورالعمل» (Prompt) مخصوص Gemma آماده شد ---")

--- «برگه دستورالعمل» (Prompt) مخصوص Gemma آماده شد ---


In [23]:
# --- دانلود فایل‌های PDF ---
print("--- در حال دانلود فایل‌های PDF ---")
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Eslahiye-PHD1405-.konkur.in.pdf" -O daftarche_asli.pdf
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Phd1405-.konkur.in.pdf" -O daftarche_eslahiye.pdf
print("--- فایل‌ها دانلود شدند ---")

--- در حال دانلود فایل‌های PDF ---
--- فایل‌ها دانلود شدند ---


In [24]:
# --- خواندن، بارگذاری و خُرد کردن متن‌ها ---
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ۱. خواندن هر دو فایل PDF
print("... در حال خواندن فایل‌های PDF")
loader_asli = PyPDFLoader("daftarche_asli.pdf")
loader_eslahiye = PyPDFLoader("daftarche_eslahiye.pdf")
all_pages = loader_asli.load() + loader_eslahiye.load()
print(f"--- مجموعا {len(all_pages)} صفحه بارگذاری شد ---")

# ۲. خُرد کردن متن‌ها به تکه‌های کوچک (Chunks)
print("... در حال خُرد کردن متن‌ها")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
text_documents = text_splitter.split_documents(all_pages)
print(f"--- متن‌ها به {len(text_documents)} تکه (Chunk) تقسیم شدند ---")

... در حال خواندن فایل‌های PDF
--- مجموعا 58 صفحه بارگذاری شد ---
... در حال خُرد کردن متن‌ها
--- متن‌ها به 206 تکه (Chunk) تقسیم شدند ---


In [25]:
# --- ساخت «بایگانی هوشمند» و «کتابدار» ---
from langchain_community.vectorstores import DocArrayInMemorySearch

# ۱. ساخت «بایگانی هوشمند» (Vectorstore)
# (این مرحله تمام تکه‌ها را به وکتور تبدیل می‌کند و سنگین‌ترین بخش است)
print("... در حال ساخت «بایگانی هوشمند» (Vectorstore)... (این مرحله زمان‌بر است)")
vectorstore = DocArrayInMemorySearch.from_documents(
    text_documents,
    embedding=embeddings # از متغیر embeddings که در سلول‌های قبلی ساختیم استفاده می‌کند
)
print("--- «بایگانی هوشمند» با موفقیت ساخته شد ---")

# ۲. ساخت «کتابدار» (Retriever)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3}) # ۳ سند مرتبط را پیدا کن
print("--- «کتابدار» (Retriever) آماده جستجو است ---")

... در حال ساخت «بایگانی هوشمند» (Vectorstore)... (این مرحله زمان‌بر است)
--- «بایگانی هوشمند» با موفقیت ساخته شد ---
--- «کتابدار» (Retriever) آماده جستجو است ---


In [26]:
# --- تعریف تابع نهایی پرسش و پاسخ ---

print("--- در حال تعریف تابع 'ask_robot' ---")

def ask_robot(question):
    """
    این تابع یک سوال می‌گیرد و تمام مراحل RAG را انجام می‌دهد.
    """
    try:
        print("... 🔎 در حال جستجو در دفترچه‌ها ...")

        # ۱. پیدا کردن تکه‌های متن مرتبط (Retrieve)
        retrieved_context_docs = retriever.invoke(question)
        # تبدیل لیست اسناد به یک متن واحد
        retrieved_context = "\n\n".join(doc.page_content for doc in retrieved_context_docs)

        print("... 🧠 در حال آماده‌سازی پاسخ ...")

        # ۲. ساخت پرامپت نهایی با «زمینه» و «سوال» (Augment)
        formatted_prompt = prompt.format(context=retrieved_context, question=question)

        # ۳. ارسال پرامپت نهایی به مدل برای تولید جواب (Generate)
        response_from_model = apply_chat_template_and_response(formatted_prompt)

        # ۴. تمیزکاری و نمایش جواب
        parsed_response = parser.parse(response_from_model)
        return parsed_response

    except Exception as e:
        return f"خطایی در پردازش رخ داد: {e}"

print("--- تابع 'ask_robot' با موفقیت ساخته شد و آماده است ---")

--- در حال تعریف تابع 'ask_robot' ---
--- تابع 'ask_robot' با موفقیت ساخته شد و آماده است ---


In [31]:
# --- سلول ۸: از ربات بپرسید! ---

# سوال خود را اینجا بنویسید
query = "سهمیه ایثارگران چقدر است؟"

# --- اجرای تابع ---
print(f"[شما]: {query}")
answer = ask_robot(query)
print(f"\n[ربات]:\n{answer}")

print("\n" + "="*50)

# برای پرسیدن سوال جدید، فقط متن query را عوض کنید و دوباره همین سلول را اجرا کنید
query_2 = "گرایش های رشته زبان و ادبیات فارسی؟"
print(f"[شما]: {query_2}")
answer_2 = ask_robot(query_2)
print(f"\n[ربات]:\n{answer_2}")

print("\n" + "="*50)

query_3 = "گرایش های علوم اقتصادي؟"
print(f"[شما]: {query_3}")
answer_3 = ask_robot(query_3)
print(f"\n[ربات]:\n{answer_3}")


print("\n" + "="*50)

query_4 = "اسم من چیه؟"
print(f"[شما]: {query_4}")
answer_4 = ask_robot(query_4)
print(f"\n[ربات]:\n{answer_4}")

[شما]: سهمیه ایثارگران چقدر است؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
ساکنان در متن، سهمیه ایثارگران به شرح زیر است: 

* جانباز 52% و بالاتر: 52%
* جانباز 52% و کمتر و همسر و فرزندان آنان: 5% 
* آزادگان و همسر و فرزندان آنان: 5%
* همسر و فرزندان رزمندگان با حداقل شش ماه حضور داوطلبانه در جبهه: 5%
* همسر و فرزندان شهدا و مفقودالاثران: 5%

[شما]: گرایش های رشته زبان و ادبیات فارسی؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[ربات]:
رشته گرايش 
کد 
ترکيب  
ترکيب دروس امتحاني بر اساس شماره دروس 
1 2 3  
آموزش زبان و ادبيات فارسی  - 1  
رشتههاي مرتبط که فارغالتحصيلان آن ميتوانند در اين مجموعه امتحاني شرکت کنند: 
زبان و ادبيات فارسی، آموزش زبان و ادبيات فارسی، مترجمي زبان و ادبيات فارسی، مطالعات شعر و داستان

[شما]: گرایش های علوم اقتصادي؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
* اقتصادسنجي
* اقتصاد پولي 
* اقتصاد اسلامي 
* اقتصاد بخش عمومي 
* توسعه اقتصادي 
* اقتصاد بينالملل 
* اقتصاد مالي 
* اقتصاد شهري و منطقهاي 
* اقتصاد منابع 
* اقتصاد بيمه 
* اقتصاد نهادگرا 
* اقتصاد رياضي 
* اقتصاد ايران

[شما]: اسم من چیه؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم.
